# Understanding `session.state` in ADK
ADK (Agent Development Kit) enables agent memory and intelligent state handling using `session.state`. This notebook explains how `session.state` works, how to read/write to it, scope, best practices, and safety rules.

## What is `session.state`?
`session.state` is a key-value scratchpad (dict-like) used by agents to remember important info across tool calls, decisions, and user inputs.

### Use Cases:
- `user:theme` → dark mode preference
- `booking_step` → track multistep process
- `cart_items` → store user choices
- `is_logged_in` → store auth status


## State Structure
Values are stored as simple key-value pairs. Keys must be **strings**. Values must be **serializable** (e.g., str, int, list of str, etc.).

```python
session.state['user:name'] = 'Mayank'
session.state['cart_items'] = ['pen', 'book']
```

## Prefixes Define Scope
| Prefix | Scope | Persistence | Example |
|--------|-------|-------------|---------|
| _(none)_ | This session | Depends on backend | `current_step` |
| `user:` | All sessions for a user | Persistent | `user:language` |
| `app:` | Global to app | Persistent | `app:version` |
| `temp:` | Current invocation only | Not saved | `temp:api_result` |

## Inject State into LLM Instructions
You can use `{key}` syntax to inject state into the LLM prompt string.
```python
agent = LlmAgent(
  instruction='Reply in {user:language}'
)
```

In [ ]:
from google.adk.agents import LlmAgent
from google.adk.agents.readonly_context import ReadonlyContext

def custom_instruction(ctx: ReadonlyContext) -> str:
    return 'Respond using {{json}} syntax with {user:name}'

agent = LlmAgent(
  name='template_demo',
  model='gemini-2.0-flash',
  instruction=custom_instruction
)

## Using `output_key` to Save Response in State
```python
agent = LlmAgent(
  name='Greeter',
  model='gemini-2.0-flash',
  instruction='Say hello',
  output_key='last_greeting'
)
# The response will be saved into `session.state['last_greeting']`
```

In [ ]:
from google.adk.sessions import InMemorySessionService
from google.adk.events import Event, EventActions
import time

session_service = InMemorySessionService()
session = await session_service.create_session('my_app', 'user1', 'session1')

delta = {
  'task_status': 'running',
  'user:login_count': 1
}
event = Event(author='system', timestamp=time.time(), actions=EventActions(state_delta=delta))
await session_service.append_event(session, event)
print(session.state)

In [ ]:
def tool_func(tool_context):
    count = tool_context.state.get('user:searches', 0)
    tool_context.state['user:searches'] = count + 1
    tool_context.state['temp:last_result'] = 'success'

## ⚠️ Avoid Unsafe State Changes
```python
# ❌ Dangerous:
session = await session_service.get_session(...)
session.state['x'] = 'value'  # Will not be tracked/persisted
```
Always update using:
- `output_key`
- `context.state`
- `EventActions.state_delta`

## ✅ Summary
- Use `session.state` to hold memory
- Keys must be strings; values must be serializable
- Use prefixes (`user:`, `app:`, `temp:`) to define scope
- Use `output_key` or `context.state` for writing
- Never modify state directly outside an event